# Email Draft Generator - Outlook Integration Workflow

This notebook provides a step-by-step interactive workflow for generating **real Outlook draft emails** from templates and Excel data. All emails are created as actual drafts in your Outlook application for maximum safety and convenience.

## Workflow Overview:
1. **Folder Selection** - Choose output folder for backup .eml files
2. **Assumptions & Custom Fields** - Configure email template and conditional content
3. **Excel File Upload** - Select and validate recipient data
4. **Attachments Setup** - Configure attachment folder and files
5. **Generate Outlook Drafts** - Create real Outlook drafts + backup files

## Key Features:
- **🎯 Real Outlook Drafts** - Creates actual drafts in your Outlook Drafts folder
- **📄 Smart Backup Files** - .eml files named by email subject for reference
- **🔒 Maximum Safety** - Draft warnings in email body, cannot be accidentally sent
- **📎 Full Attachment Support** - Attachments are properly added to Outlook drafts
- **⚡ Seamless Workflow** - Review and send directly from Outlook

## Requirements:
- **Microsoft Outlook** installed and configured
- **Python pywin32 package**: `pip install pywin32` 
- If pywin32 is not available, falls back to .eml files only

In [1]:
# Setup - Import required libraries
import os
import sys
from pathlib import Path

print("✅ Basic imports successful")

# Import pandas
try:
    import pandas as pd
    print("✅ Pandas imported")
except ImportError:
    print("❌ Pandas not available - please install with: pip install pandas openpyxl")

# Check for Outlook integration capability
try:
    import win32com.client
    print("✅ Outlook integration available (pywin32)")
    OUTLOOK_AVAILABLE = True
except ImportError:
    print("⚠️ Outlook integration not available")
    print("   To create real Outlook drafts, install: pip install pywin32")
    print("   Without this, only .eml backup files will be created")
    OUTLOOK_AVAILABLE = False

# Initialize basic variables
current_dir = Path.cwd()
print(f"📁 Working directory: {current_dir}")

print("🎯 Setup complete!")
if OUTLOOK_AVAILABLE:
    print("🚀 Ready to create Outlook drafts + backup files")
else:
    print("📄 Ready to create .eml backup files only")

✅ Basic imports successful
✅ Pandas imported
✅ Pandas imported
✅ Outlook integration available (pywin32)
📁 Working directory: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main
🎯 Setup complete!
🚀 Ready to create Outlook drafts + backup files
✅ Outlook integration available (pywin32)
📁 Working directory: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main
🎯 Setup complete!
🚀 Ready to create Outlook drafts + backup files


## 1. Folder Selection

Configure where you want to generate the email files. The cell below will:
- Use the default `generated_emails` folder in the current directory
- Or you can edit the `output_folder` variable to specify a custom path
- The folder will be created automatically if it doesn't exist

In [2]:
# Load email generator module
import json
import importlib
import inspect

# Import the email generator module
try:
    import email_file_generator as efg
    # Reload the module to ensure we have the latest version
    importlib.reload(efg)
    print("✅ Email file generator module loaded and reloaded")
    
    # Verify the main function has the create_eml_backup parameter
    sig = inspect.signature(efg.main)
    params = list(sig.parameters.keys())
    print(f"📋 Main function parameters: {params}")
    
    if 'create_eml_backup' in params:
        print("✅ create_eml_backup parameter found")
    else:
        print("❌ create_eml_backup parameter missing - there may be a module import issue")
        
except ImportError:
    print("❌ email_file_generator.py not found in current directory")
    print("Make sure this notebook is in the same folder as email_file_generator.py")

print("🚀 All modules loaded - ready for workflow!")

✅ Email file generator module loaded and reloaded
📋 Main function parameters: ['template_path', 'excel_path', 'attachments_dir', 'output_dir', 'conditionals_path', 'use_outlook', 'create_eml_backup']
✅ create_eml_backup parameter found
🚀 All modules loaded - ready for workflow!


In [3]:
# 1. Folder Selection - Direct Input
from pathlib import Path

# Set your output folder path here
# Option 1: Use default folder
output_folder = Path.cwd() / "generated_emails"

# Option 2: Or specify a custom path (uncomment and modify the line below)
# output_folder = Path(r"C:\Your\Custom\Path\emails")

# Create the folder if it doesn't exist
output_folder.mkdir(exist_ok=True)

print(f"✅ Output folder set to: {output_folder}")
print(f"📁 Folder exists: {output_folder.exists()}")
print()
print("To change the output folder:")
print("1. Edit the 'output_folder' variable above")
print("2. Rerun this cell")
print()
print("Current working directory:", Path.cwd())

✅ Output folder set to: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main\generated_emails
📁 Folder exists: True

To change the output folder:
1. Edit the 'output_folder' variable above
2. Rerun this cell

Current working directory: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main


## 2. Assumptions & Custom Fields Configuration

Configure your email template and conditional content settings.

In [4]:
# 2. Email Template Configuration - Direct Input
import json

# Set your template path here
# Option 1: Use default template
template_path = Path.cwd() / "email_template.txt"

# Option 2: Or specify a custom template path (uncomment and modify)
# template_path = Path(r"C:\Your\Custom\Path\your_template.txt")

# Check if template exists, if not create a basic one
if not template_path.exists():
    print("📝 Creating basic email template...")
    basic_template = """Dear [FirstName],

Thank you for your interest in our services.

[Conditional:SpecialOffer]

If you have any questions, please don't hesitate to contact us.

Best regards,
[SenderName]
[Company]"""
    
    with open(template_path, 'w', encoding='utf-8') as f:
        f.write(basic_template)
    print(f"✅ Created template at: {template_path}")

# Load and display template
try:
    with open(template_path, 'r', encoding='utf-8') as f:
        template_content = f.read()
    
    # Extract variables from template
    variables = efg.extract_variables(template_content)
    
    print(f"✅ Template loaded: {template_path}")
    print(f"📝 Template variables found: {', '.join(variables) if variables else 'None'}")
    print()
    print("Template content:")
    print("-" * 50)
    print(template_content)
    print("-" * 50)
    
except Exception as e:
    print(f"❌ Error loading template: {e}")
    print("Please check the template path and try again.")

print()
print("To use a different template:")
print("1. Edit the 'template_path' variable above")
print("2. Rerun this cell")

✅ Template loaded: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main\email_template.txt
📝 Template variables found: FirstName

Template content:
--------------------------------------------------
[FirstName], 

Hope all is well. I’m reaching out regarding a new opportunity to partner with Forge52 Capital Partners on a c. ~$70m direct capital raise (~$55m from co-investors) in connection with their acquisition of a premier provider of corrosion-resistant process equipment. We’ve included some of the key investments highlights below but have also attached a teaser to provide some additional information.

If this opportunity is of interest, we’ve also attached the form NDA for your review. Upon execution, we would be happy to provide VDR access, which includes a comprehensive transaction overview, financial model, and fulsome standard and business diligence materials.

Please let us know your availability over the coming days for a brief call to provide additional b

In [5]:
# Conditional Lines Configuration - Direct Input
conditionals_path = Path.cwd() / "conditional_lines.json"

# Check if conditional lines file exists, if not create basic ones
if not conditionals_path.exists():
    print("📋 Creating basic conditional lines...")
    conditional_lines = {
        "SpecialOffer": "As a valued customer, we would like to offer you a special discount of [Discount]%.",
        "MeetingReminder": "This is a friendly reminder about our meeting scheduled for [MeetingDate].",
        "BrochureInfo": "Please see the attached brochure for more information about our new products."
    }
    
    with open(conditionals_path, 'w', encoding='utf-8') as f:
        json.dump(conditional_lines, f, indent=2)
    print(f"✅ Created conditional lines at: {conditionals_path}")
else:
    print(f"📋 Loading existing conditional lines from: {conditionals_path}")

# Load conditional lines
try:
    with open(conditionals_path, 'r', encoding='utf-8') as f:
        conditional_lines = json.load(f)
    
    print(f"✅ Loaded {len(conditional_lines)} conditional lines:")
    for key, value in conditional_lines.items():
        print(f"  • {key}: {value}")
        
except Exception as e:
    print(f"❌ Error loading conditional lines: {e}")
    conditional_lines = {}

print()
print("Conditional lines allow optional text based on flags in your Excel file.")
print("To modify conditional lines, edit the conditional_lines.json file and rerun this cell.")

📋 Loading existing conditional lines from: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main\conditional_lines.json
✅ Loaded 3 conditional lines:
  • SpecialOffer: As a valued customer, we would like to offer you a special discount.
  • MeetingReminder: This is a friendly reminder about our meeting scheduled for [MeetingDate].
  • BrochureInfo: Please see the attached brochure for more information about our new products.

Conditional lines allow optional text based on flags in your Excel file.
To modify conditional lines, edit the conditional_lines.json file and rerun this cell.


## 3. Excel File Upload & Validation

Select your Excel file containing recipient data and validate the columns.

In [6]:
# 3. Excel File Selection and Validation - Direct Input
import pandas as pd

# Set your Excel file path here
# Option 1: Use default recipients file
excel_path = Path.cwd() / "recipients.xlsx"

# Option 2: Or specify a custom Excel file path (uncomment and modify)
# excel_path = Path(r"C:\Your\Custom\Path\your_recipients.xlsx")

# Load and validate Excel file
try:
    df = pd.read_excel(excel_path)
    
    print(f"✅ Excel file loaded: {excel_path}")
    print(f"📊 File info:")
    print(f"  • Rows: {len(df)}")
    print(f"  • Columns: {len(df.columns)}")
    print(f"  • Column names: {', '.join(df.columns.tolist())}")
    print()
    
    # Validate required columns
    email_cols = [col for col in df.columns if col.lower() in ['email', 'to']]
    attachment_cols = [col for col in df.columns if col.lower().startswith('attachment')]
    
    print("📋 Column validation:")
    if email_cols:
        print(f"  ✅ Email column found: {email_cols[0]}")
    else:
        print("  ❌ No Email/To column found - this is required!")
    
    if attachment_cols:
        print(f"  📎 Attachment columns: {', '.join(attachment_cols)}")
    else:
        print("  ℹ️ No attachment columns found")
    
    # Check for conditional line columns
    conditional_cols = [col for col in df.columns if col in conditional_lines.keys()]
    if conditional_cols:
        print(f"  🔀 Conditional columns: {', '.join(conditional_cols)}")
    else:
        print("  ℹ️ No conditional line columns found")
    
    print()
    print("📝 First 5 rows preview:")
    print(df.head().to_string())
    
except FileNotFoundError:
    print(f"❌ Excel file not found: {excel_path}")
    print()
    print("To fix this:")
    print("1. Make sure your Excel file exists")
    print("2. Update the 'excel_path' variable above with the correct path")
    print("3. Rerun this cell")
    df = None
    
except Exception as e:
    print(f"❌ Error loading Excel file: {e}")
    df = None

print()
print("To use a different Excel file:")
print("1. Edit the 'excel_path' variable above")
print("2. Rerun this cell")

✅ Excel file loaded: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main\recipients.xlsx
📊 File info:
  • Rows: 187
  • Columns: 9
  • Column names: Approached, Firm, Simplified firm, Email, CC, From, Subject, FirstName, Attachment

📋 Column validation:
  ✅ Email column found: Email
  📎 Attachment columns: Attachment
  ℹ️ No conditional line columns found

📝 First 5 rows preview:
    Approached             Firm  Simplified firm                                                                 Email                                       CC                          From                                    Subject         FirstName  Attachment
0  To approach    1248 Holdings    1248 Holdings                                                   creighton@12-48.com  cl-projecttitanium@campbell-lutyens.com  susetka@campbell-lutyens.com    Project Titanium Launch – 1248 Holdings         Creighton         NaN
1  To approach  1888 Management  1888 Management                      

## 4. Attachments Setup

Configure the attachments folder and verify attachment files.

In [7]:
# 4. Attachments Setup - Direct Input

# Set your attachments folder path here
# Option 1: Use default attachments folder
attachments_dir = Path.cwd() / "attachments"

# Option 2: Or specify a custom attachments folder (uncomment and modify)
# attachments_dir = Path(r"C:\Your\Custom\Path\attachments")

# Check attachments folder
if attachments_dir.exists():
    files = list(attachments_dir.iterdir())
    print(f"✅ Attachments folder found: {attachments_dir}")
    print(f"📎 Files in folder: {len(files)}")
    
    if files:
        print("📁 Files found:")
        for file in files[:10]:  # Show first 10 files
            print(f"  • {file.name}")
        if len(files) > 10:
            print(f"  ... and {len(files) - 10} more files")
    else:
        print("  ℹ️ Folder is empty")
    
    # Validate attachments if Excel is loaded
    if df is not None:
        attachment_columns = [c for c in df.columns if c.lower().startswith('attachment')]
        if attachment_columns:
            print()
            print("🔍 Attachment validation:")
            missing_files = []
            found_files = []
            
            for col in attachment_columns:
                unique_files = df[col].dropna().unique()
                print(f"  Column '{col}': {len(unique_files)} unique files")
                
                for filename in unique_files:
                    file_path = attachments_dir / str(filename)
                    if file_path.exists():
                        found_files.append(filename)
                    else:
                        missing_files.append(filename)
            
            if found_files:
                print(f"  ✅ Found files: {', '.join(found_files[:5])}")
                if len(found_files) > 5:
                    print(f"      ... and {len(found_files) - 5} more")
            
            if missing_files:
                print(f"  ❌ Missing files: {', '.join(missing_files[:5])}")
                if len(missing_files) > 5:
                    print(f"      ... and {len(missing_files) - 5} more")
        else:
            print("  ℹ️ No attachment columns found in Excel file")
else:
    print(f"❌ Attachments folder not found: {attachments_dir}")
    print("This folder will be created if needed, or you can create it manually.")

print()
print("To use a different attachments folder:")
print("1. Edit the 'attachments_dir' variable above")
print("2. Rerun this cell")

✅ Attachments folder found: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main\attachments
📎 Files in folder: 2
📁 Files found:
  • NDA Project Titanium .docx
  • Project Titanium - Transaction Teaser.pdf

🔍 Attachment validation:
  Column 'Attachment': 0 unique files

To use a different attachments folder:
1. Edit the 'attachments_dir' variable above
2. Rerun this cell


## 5. Preview & Generate Emails

Preview a sample email and then generate all email files.

In [8]:
# 5. Email Preview and Generation - Direct Execution

# Check if all required components are ready
print("🔍 Checking configuration...")

ready_to_generate = True
issues = []

if not globals().get('output_folder') or not output_folder:
    issues.append("❌ Output folder not set")
    ready_to_generate = False
else:
    print(f"✅ Output folder: {output_folder}")

if not globals().get('template_path') or not template_path or not template_path.exists():
    issues.append("❌ Template file not found")
    ready_to_generate = False
else:
    print(f"✅ Template file: {template_path}")

if not globals().get('excel_path') or not excel_path or not excel_path.exists():
    issues.append("❌ Excel file not found")
    ready_to_generate = False
else:
    print(f"✅ Excel file: {excel_path}")

# Check if DataFrame exists and is not None
df_exists = globals().get('df') is not None
if not df_exists:
    issues.append("❌ Excel data not loaded")
    ready_to_generate = False
else:
    print(f"✅ Excel data loaded: {len(df)} rows")

if not globals().get('attachments_dir'):
    attachments_dir = Path.cwd() / "attachments"

print(f"✅ Attachments folder: {attachments_dir}")

if issues:
    print("\n🚨 Issues found:")
    for issue in issues:
        print(f"  {issue}")
    print("\nPlease fix the issues above and rerun this cell.")
else:
    print("\n🎯 All components ready!")

# Preview first email if everything is ready
if ready_to_generate:
    print("\n" + "="*60)
    print("📧 EMAIL PREVIEW (First Row)")
    print("="*60)
    
    try:
        # Load template
        with open(template_path, 'r', encoding='utf-8') as f:
            template_text = f.read()
        
        # Get first row for preview
        row = df.iloc[0]
        
        # Extract variables
        variables = efg.extract_variables(template_text)
        
        # Create email message for preview
        attachment_columns = [c for c in df.columns if c.lower().startswith('attachment')]
        
        msg = efg.create_email_message(
            row, template_text, variables, 
            attachments_dir, attachment_columns, conditional_lines
        )
        
        # Display preview
        print(f"To: {msg.get('To', 'No recipient')}")
        print(f"CC: {msg.get('CC', 'None')}")
        print(f"From: {msg.get('From', 'sender@example.com')}")
        print(f"Subject: {msg.get('Subject', 'No Subject')}")
        print()
        print("Body:")
        print("-" * 40)
        print(msg.get_content())
        print("-" * 40)
        
        print("\n✅ Email preview generated successfully!")
        
    except Exception as e:
        print(f"\n❌ Error generating preview: {e}")
        ready_to_generate = False

print("\n" + "="*60)
print("🚀 GENERATE ALL EMAILS")
print("="*60)

if ready_to_generate:
    print("Ready to generate emails. Run the next cell to start generation.")
else:
    print("Fix the issues above before generating emails.")

🔍 Checking configuration...
✅ Output folder: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main\generated_emails
✅ Template file: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main\email_template.txt
✅ Excel file: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main\recipients.xlsx
✅ Excel data loaded: 187 rows
✅ Attachments folder: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main\attachments

🎯 All components ready!

📧 EMAIL PREVIEW (First Row)
  ✅ Attached: NDA Project Titanium .docx
  ✅ Attached: Project Titanium - Transaction Teaser.pdf
To: creighton@12-48.com
CC: cl-projecttitanium@campbell-lutyens.com, <>
From: susetka@campbell-lutyens.com
Subject: Project Titanium Launch – 1248 Holdings

Body:
----------------------------------------

❌ Error generating preview: 'multipart/mixed'

🚀 GENERATE ALL EMAILS
Fix the issues above before generating emails.


In [10]:
# Generate All Emails - Run this cell to create Outlook drafts only (no .eml files)

print("🚀 Starting email generation...")
print("="*60)

# Test Outlook connection first if pywin32 is available
use_outlook = True
if OUTLOOK_AVAILABLE:
    print("🔍 Testing Outlook connection...")
    try:
        import win32com.client
        outlook = win32com.client.Dispatch("Outlook.Application")
        test_mail = outlook.CreateItem(0)
        test_mail.Subject = "Connection Test"
        test_mail.Body = "Test"
        # Don't save the test email, just test the connection
        print("✅ Outlook connection successful!")
        del test_mail
        del outlook
    except Exception as e:
        print(f"❌ Outlook connection failed: {e}")
        print("⚠️ Falling back to .eml files only")
        use_outlook = False
else:
    use_outlook = False
    print("ℹ️ Using .eml files only (pywin32 not available)")

try:
    # Ensure output folder exists
    output_folder.mkdir(exist_ok=True)
    
    # Set up paths for generation
    conditionals_file = Path.cwd() / "conditional_lines.json"
    
    print(f"\n{'📧 Creating Outlook drafts only' if use_outlook else '📄 Creating .eml backup files'}")
    print("-" * 50)
    
    # Generate emails using the main function - only create Outlook drafts, no .eml backups
    efg.main(
        template_path=str(template_path),
        excel_path=str(excel_path),
        attachments_dir=str(attachments_dir),
        output_dir=str(output_folder),
        conditionals_path=str(conditionals_file) if conditionals_file.exists() else None,
        use_outlook=use_outlook,  # Use result of connection test
        create_eml_backup=False   # Only create Outlook drafts, no .eml files
    )
    
    # Count generated backup files (should be 0 for Outlook-only mode)
    generated_files = list(output_folder.glob("*.eml"))
    
    print("\n" + "="*60)
    print("🎉 EMAIL GENERATION COMPLETED!")
    print("="*60)
    
    if use_outlook:
        print("✅ Check your Outlook Drafts folder for draft emails")
        print("📧 All emails created as Outlook drafts only (no .eml backup files)")
    else:
        print(f"📄 Backup .eml files created: {len(generated_files)}")
        print(f"📁 Backup folder: {output_folder}")
    
    if generated_files and not use_outlook:
        print("\n📋 Backup files (named by subject):")
        for file in generated_files[:5]:  # Show first 5 files
            print(f"  • {file.name}")
        
        if len(generated_files) > 5:
            print(f"  ... and {len(generated_files) - 5} more files")
    
    print(f"\n🎯 Next Steps:")
    if use_outlook:
        print(f"  1. 📧 Open Outlook and check your Drafts folder")
        print(f"  2. 👀 Review each draft email carefully") 
        print(f"  3. 📤 Send when ready!")
        print(f"\n💡 No .eml backup files were created - emails exist only as Outlook drafts")
    else:
        print(f"  1. 📄 Review the .eml backup files")
        print(f"  2. 📧 Import them into your email client")
        print(f"  3. ✏️ Edit and send manually")
    
except Exception as e:
    print(f"\n❌ Error generating emails: {e}")
    print("Please check your configuration and try again.")
    print("\n💡 Common solutions:")
    print("  • Ensure Outlook is installed and configured")
    print("  • Try running Outlook manually first")
    print("  • Check if antivirus software is blocking the connection")
    print("  • Restart this notebook if needed")
    
    # Offer fallback option with .eml files
    print(f"\n🔄 Want to try .eml files as backup? Run this:")
    print(f"   efg.main(template_path=str(template_path), excel_path=str(excel_path),")
    print(f"            attachments_dir=str(attachments_dir), output_dir=str(output_folder),")
    print(f"            use_outlook=False, create_eml_backup=True)")

🚀 Starting email generation...
🔍 Testing Outlook connection...
✅ Outlook connection successful!

📧 Creating Outlook drafts only
--------------------------------------------------
Loaded conditional lines: ['SpecialOffer', 'MeetingReminder', 'BrochureInfo']
Found variables: ['FirstName', 'MeetingDate']
📧 Creating Outlook drafts only...
  📧 Connecting to Outlook... ✅
  📝 Creating email item...📧 Creating Outlook drafts only...
  📧 Connecting to Outlook... ✅
  📝 Creating email item... ✅
  📄 Processing email body... ✅
  📄 Processing email body... ✅
  📎 Adding attachments... ✅
  📎 Adding attachments...
    Trying to attach: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main\attachments\NDA Project Titanium .docx
    ✅ Successfully attached: NDA Project Titanium .docx
    Trying to attach: c:\Users\tuncel\Documents\VSCode\autoemlbuilder-main\autoemlbuilder-main\attachments\Project Titanium - Transaction Teaser.pdf
    ✅ Successfully attached: Project Titanium - Transacti

## Summary

This streamlined workflow notebook allows you to generate **Outlook Draft Emails** directly:

1. **Setup** - Import required libraries and modules
2. **Configure Output Folder** - Set where to save generated files (if needed for fallback)
3. **Configure Email Template** - Set up email template with variables and conditional content
4. **Load Excel Data** - Import and validate recipient information from Excel file
5. **Setup Attachments** - Configure attachment files and validation
6. **Preview & Generate** - Preview sample email and generate all Outlook draft emails

### Key Features:
- **? Outlook Drafts Only** - Creates real drafts directly in your Outlook Drafts folder
- **? No .eml Backup Files** - Streamlined workflow without cluttering your file system
- **? Auto-Attach All Files** - Automatically attaches all files in the attachments folder
- **📋 Default CC** - Automatically CCs '[' on all emails
- **🎯 HTML Email Support** - Rich formatting with **bold**, *italic*, and __underline__ text
- **⚡ Direct Workflow** - From data to Outlook drafts in one step

### Standard Features:
- **Simple Direct Configuration** - Edit variables directly in cells, no complex UI
- **Auto-creation** - Creates template and conditional files if they don't exist
- **Real-time Validation** - Checks files, folders, and data before generation
- **Clear Error Messages** - Shows exactly what needs to be fixed
- **Email Preview** - See how the first email will look before generating all
- **Support for Advanced Features**:
  - Conditional content based on Excel flags
  - Multiple recipients (To, CC, BCC)
  - Automatic file attachments from folder
  - Variable substitution
  - HTML formatting support

### How to Use:
1. **Run cells 2-4** - Setup and load modules
2. **Run cell 5** - Configure output folder (edit path if needed)
3. **Run cell 7** - Configure email template (edit path if needed)
4. **Run cell 8** - Load conditional lines configuration
5. **Run cell 10** - Load Excel file (edit path if needed)
6. **Run cell 12** - Configure attachments folder
7. **Run cell 14** - Preview and validate everything
8. **Run cell 15** - Generate all Outlook draft emails

### Generated Outlook Drafts:
The system creates **real Outlook drafts** that:
- **Appear directly in your Outlook Drafts folder** for immediate review
- **Include all personalized content** based on Excel data
- **Have conditional text** based on flags in Excel columns
- **Include all file attachments** from the attachments folder automatically
- **Have proper email headers** (To, CC, BCC, Subject, From)
- **Support HTML formatting** for professional appearance
- **Automatically CC '['** as specified

### Automatic Features:
- **📎 All Files Attached** - Every file in the attachments folder is automatically attached to every email
- **📧 Default CC** - '[' is automatically added to the CC field of every email
- **🎨 HTML Formatting** - Use **bold**, *italic*, __underline__ in your templates
- **🔍 Smart Debugging** - See exactly which files are being attached during generation

### Next Steps After Generation:
1. **📧 Open Outlook and check your Drafts folder**
2. **👀 Review each draft email carefully**
3. **✏️ Make any final edits needed**
4. **📤 Send when ready!**

**? Streamlined Workflow:** This system creates production-ready Outlook drafts without cluttering your file system with backup files. Everything happens directly in Outlook for the most efficient workflow possible.